In [9]:
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import pandas as pd
import numpy as np

In [11]:
# File path
file_path = "/Users/freewheelin/Documents/ev_charging_station.txt"

# Load the dataset
data = pd.read_csv(file_path)

print(data.head())

   sessionId  kwhTotal  dollars              created                ended  \
0    1366563      7.78     0.00  0014-11-18 15:40:26  0014-11-18 17:11:04   
1    3075723      9.74     0.00  0014-11-19 17:40:26  0014-11-19 19:51:04   
2    4228788      6.76     0.58  0014-11-21 12:05:46  0014-11-21 16:46:04   
3    3173284      6.17     0.00  0014-12-03 19:16:12  0014-12-03 21:02:18   
4    3266500      0.93     0.00  0014-12-11 20:56:11  0014-12-11 21:14:06   

   startTime  endTime  chargeTimeHrs weekday platform  ...  managerVehicle  \
0         15       17       1.510556     Tue  android  ...               0   
1         17       19       2.177222     Wed  android  ...               0   
2         12       16       4.671667     Fri  android  ...               0   
3         19       21       1.768333     Wed  android  ...               0   
4         20       21       0.298611     Thu  android  ...               0   

   facilityType  Mon  Tues  Wed  Thurs  Fri  Sat  Sun  reportedZip  

In [85]:
# Define the total number of unique userIds and locationIds
user_encoder = LabelEncoder()
location_encoder = LabelEncoder()

# Transform the `userId` and `locationId` so they contain only numbers ranging from 0 to len(userId)
data['userId'] = user_encoder.fit_transform(data['userId'])
data['locationId'] = location_encoder.fit_transform(data['locationId'])

num_users = len(user_encoder.classes_)
num_locations = len(location_encoder.classes_)

# Define embedding layers
user_embedding_dim = 8  # You can choose any dimension
location_embedding_dim = 8  # You can choose any dimension

# create the embedding layers
user_embedding_layer = tf.keras.layers.Embedding(input_dim=num_users, output_dim=user_embedding_dim, name='user_embedding')
location_embedding_layer = tf.keras.layers.Embedding(input_dim=num_locations, output_dim=location_embedding_dim, name='location_embedding')

### normalize the continuous variables
# List of continuous features
continuous_features = ['chargeTimeHrs', 'startTime', 'endTime']

# Normalize the features
scaler = MinMaxScaler()
X_continuous_features = scaler.fit_transform(data[continuous_features])

# List of one-hot-encoded weekday columns
one_hot_weekdays = ['Mon', 'Tues', 'Wed', 'Thurs', 'Fri', 'Sat', 'Sun']

# Target variable
y = data['kwhTotal'].values

In [358]:
# this one seems to be working for some reason
# Filter required columns
columns_to_keep = ['kwhTotal', 'chargeTimeHrs', 
                   'userId', 'locationId', 
                   'Mon', 'Tues', 'Wed', 'Thurs', 'Fri', 'Sat', 'Sun']
data = data[columns_to_keep]

# Normalize numeric columns
scaler = MinMaxScaler()
numeric_columns = ['kwhTotal', 'chargeTimeHrs']
data[numeric_columns] = scaler.fit_transform(data[numeric_columns])

# Prepare one-hot columns
one_hot_columns = ['Mon', 'Tues', 'Wed', 'Thurs', 'Fri', 'Sat', 'Sun']
one_hot_values = data[one_hot_columns].values

# Create embeddings for `userId` and `locationId`
unique_user_ids = data['userId'].unique()
unique_location_ids = data['locationId'].unique()

user_id_lookup = {id: idx for idx, id in enumerate(unique_user_ids)}
location_id_lookup = {id: idx for idx, id in enumerate(unique_location_ids)}

data['userId'] = data['userId'].map(user_id_lookup)
data['locationId'] = data['locationId'].map(location_id_lookup)

# Define embedding layers
embedding_dim = 8  # Example dimension
user_embedding = tf.keras.layers.Embedding(input_dim=len(unique_user_ids), output_dim=embedding_dim)
location_embedding = tf.keras.layers.Embedding(input_dim=len(unique_location_ids), output_dim=embedding_dim)

# Create TensorFlow dataset
def preprocess(userId, locationId, numeric, one_hot):
    user_embed = user_embedding(userId)
    location_embed = location_embedding(locationId)
    combined = tf.concat([user_embed, location_embed, numeric, one_hot], axis=-1)
    return combined

# Convert data to tensors
user_ids = tf.constant(data['userId'].values, dtype=tf.int32)
location_ids = tf.constant(data['locationId'].values, dtype=tf.int32)
numeric_features = tf.constant(data[numeric_columns].values, dtype=tf.float32)
one_hot_features = tf.constant(one_hot_values, dtype=tf.float32)

# Zip into a dataset
dataset = tf.data.Dataset.from_tensor_slices((user_ids, location_ids, numeric_features, one_hot_features))
dataset = dataset.map(preprocess)

# Verify the shape of combined tensors
#for x in dataset.take(1):
#    print(x.shape)  # Should match the final feature vector dimensions

(25,)


2024-12-18 13:04:43.123896: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [51]:
# these one works with batches, seems to be working too
# Load your dataset
# Assume `data` is a pandas DataFrame with the required columns

# Step 1: Normalize numeric columns
scaler = MinMaxScaler()
numeric_columns = ['kwhTotal', 'chargeTimeHrs']
data[numeric_columns] = scaler.fit_transform(data[numeric_columns])

# Step 2: Prepare one-hot encoded columns
one_hot_columns = ['Mon', 'Tues', 'Wed', 'Thurs', 'Fri', 'Sat', 'Sun']
one_hot_values = data[one_hot_columns].values

# Step 3: Create mappings for `userId` and `locationId`
unique_user_ids = data['userId'].unique()
unique_location_ids = data['locationId'].unique()

user_id_lookup = {id: idx for idx, id in enumerate(unique_user_ids)}
location_id_lookup = {id: idx for idx, id in enumerate(unique_location_ids)}

data['userId'] = data['userId'].map(user_id_lookup)
data['locationId'] = data['locationId'].map(location_id_lookup)

# Convert processed columns to tensors
user_ids = tf.constant(data['userId'].values, dtype=tf.int32)
location_ids = tf.constant(data['locationId'].values, dtype=tf.int32)
numeric_features = tf.constant(data[numeric_columns].values, dtype=tf.float32)
one_hot_features = tf.constant(one_hot_values, dtype=tf.float32)

# Combine all features into a dictionary for batching
features = tf.data.Dataset.from_tensor_slices({
    'userId': user_ids,
    'locationId': location_ids,
    'numeric_features': numeric_features,
    'one_hot_features': one_hot_features
})

# Target variable
y = data['kwhTotal'].values

# Convert target variable to a tensor
y_tensor = tf.constant(y, dtype=tf.float32)

# Zip features with the target variable
dataset = tf.data.Dataset.zip((features, tf.data.Dataset.from_tensor_slices(y_tensor)))

# --------------------------------------------
# Step 4: Define batch size and batch the data
batch_size = 32
batched_dataset = dataset.batch(batch_size)

# Step 5: Define embedding layers
user_embedding_dim = 16
location_embedding_dim = 16

user_embedding_layer = tf.keras.layers.Embedding(input_dim=len(unique_user_ids), output_dim=user_embedding_dim)
location_embedding_layer = tf.keras.layers.Embedding(input_dim=len(unique_location_ids), output_dim=location_embedding_dim)

# Process batches
for batch, y in batched_dataset:
    # Extract batch features
    user_id_batch = batch['userId']
    location_id_batch = batch['locationId']
    numeric_batch = batch['numeric_features']
    one_hot_batch = batch['one_hot_features']
    
    # Compute embeddings
    user_embeddings = user_embedding_layer(user_id_batch)
    location_embeddings = location_embedding_layer(location_id_batch)
    
    # Concatenate all features
    concatenated_features = tf.concat([user_embeddings, location_embeddings, numeric_batch, one_hot_batch], axis=-1)
    print(concatenated_features)
    
    # Example forward pass (1 hidden layer)
    W1 = tf.random.normal(shape=(concatenated_features.shape[-1], 64))  # Example weight matrix
    b1 = tf.random.normal(shape=(64,))  # Example bias
    hidden_layer = tf.nn.relu(tf.linalg.matmul(concatenated_features, W1) + b1)
    
    #print("Hidden Layer Output for this batch:", hidden_layer)

tf.Tensor(
[[-0.01244352  0.01136253  0.01784826 ...  0.          0.
   0.        ]
 [-0.01244352  0.01136253  0.01784826 ...  0.          0.
   0.        ]
 [-0.01244352  0.01136253  0.01784826 ...  1.          0.
   0.        ]
 ...
 [-0.01244352  0.01136253  0.01784826 ...  0.          1.
   0.        ]
 [-0.01244352  0.01136253  0.01784826 ...  0.          0.
   0.        ]
 [-0.01244352  0.01136253  0.01784826 ...  0.          0.
   0.        ]], shape=(32, 41), dtype=float32)
tf.Tensor(
[[-0.01244352  0.01136253  0.01784826 ...  0.          0.
   0.        ]
 [-0.01244352  0.01136253  0.01784826 ...  0.          0.
   0.        ]
 [-0.01244352  0.01136253  0.01784826 ...  0.          0.
   0.        ]
 ...
 [-0.01244352  0.01136253  0.01784826 ...  0.          0.
   0.        ]
 [-0.01244352  0.01136253  0.01784826 ...  0.          0.
   0.        ]
 [-0.01244352  0.01136253  0.01784826 ...  0.          0.
   0.        ]], shape=(32, 41), dtype=float32)
tf.Tensor(
[[-0.01244352  

2024-12-21 21:30:22.423029: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [59]:
def model(X_train, Y_train, X_test, Y_test, learning_rate = 0.0001,
          num_epochs = 1500, minibatch_size = 32, print_cost = True):
    """
    """

    train_acc = []
    test_acc = []
    costs = []

    parameters = initialize_parameters()

    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    W3 = parameters['W3']
    b3 = parameters['b3']

    # define the optimizer
    optimizer_defined = tf.keras.optimizers.Adam(learning_rate)

    # keep track of model accuracy
    test_accuracy = tf.keras.metrics.CategoricalAccuracy()
    train_accuracy = tf.keras.metrics.CategoricalAccuracy()

    # zip the data into a tuple of (input, expected label)
    #dataset = tf.data.Dataset.zip((X_train, Y_train))
    #test_dataset = tf.data.Dataset.zip((X_test, Y_test))

    # get the number of elements of dataset using the cardiality method
    m = dataset.cardinality().numpy()

    # form minibatches from the dataset
    minibatches = dataset.batch(minibatch_size).prefetch(8)
    #test_minibatches = test_dataset.batch(minibatch_size).prefetch(8)

    for epoch in range(num_epochs):
        epoch_cost = 0

        # we need to restart from 0 when start a new epoch
        train_accuracy.reset_state()
        
        for minibatch_X, minibatch_Y in minibatches:
            # Extract batch features
            user_id_batch = minibatch_X['userId']
            location_id_batch = minibatch_X['locationId']
            numeric_batch = minibatch_X['numeric_features']
            one_hot_batch = minibatch_X['one_hot_features']
    
            # Compute embeddings
            user_embeddings = user_embedding_layer(user_id_batch)
            location_embeddings = location_embedding_layer(location_id_batch)
    
            # Concatenate all features
            concatenated_features = tf.concat([user_embeddings, location_embeddings, numeric_batch, one_hot_batch], axis=-1)
            
            with tf.GradientTape() as tape:
                # predict
                z3 = forward_propagation(tf.transpose(concatenated_features), parameters)

                # cost
                minibatch_cost = compute_cost(z3, tf.transpose(minibatch_Y))
                #print(minibatch_Y.shape, z3.shape)

            # update the accuracy of all batches
            train_accuracy.update_state(tf.transpose(z3), minibatch_Y)

            # now use the gradient tape to get the gradients and update the parameters
            trainable_variables = [W1, b1, W2, b2, W3, b3]
            grads = tape.gradient(minibatch_cost, trainable_variables)
            optimizer_defined.apply_gradients(zip(grads, trainable_variables))

            # add all the costs
            epoch_cost += minibatch_cost

        # we devide the cost of each epoch by number of samples to get the cost per sample
        epoch_cost /= m

        # print cost every 10 epoch
        if print_cost == True and epoch % 10 == 0:
            print("cost after epoch %i: %f" % (epoch, epoch_cost))
            print("train accuracy:", train_accuracy.result())

            for (minibatch_X, minibatch_Y) in test_minibatches:
                Z3 = forward_propagation(tf.transpose(minibatch_X), parameters)
                test_accuracy.update_state(tf.transpose(Z3), minibatch_Y)
            print("Test_accuracy:", test_accuracy.result())

            costs.append(epoch_cost)
            train_acc.append(train_accuracy.result())
            test_acc.append(test_accuracy.result())
            test_accuracy.reset_state() # this resets every 10 epoch

    return parameters, costs, train_acc, test_acc

In [61]:
data_iter = iter(dataset)
for i in range(10):
    print(next(data_iter))

({'userId': <tf.Tensor: shape=(), dtype=int32, numpy=0>, 'locationId': <tf.Tensor: shape=(), dtype=int32, numpy=0>, 'numeric_features': <tf.Tensor: shape=(2,), dtype=float32, numpy=array([0.3285473 , 0.02712613], dtype=float32)>, 'one_hot_features': <tf.Tensor: shape=(7,), dtype=float32, numpy=array([0., 1., 0., 0., 0., 0., 0.], dtype=float32)>}, <tf.Tensor: shape=(), dtype=float32, numpy=0.32854729890823364>)
({'userId': <tf.Tensor: shape=(), dtype=int32, numpy=0>, 'locationId': <tf.Tensor: shape=(), dtype=int32, numpy=0>, 'numeric_features': <tf.Tensor: shape=(2,), dtype=float32, numpy=array([0.41131756, 0.03919784], dtype=float32)>, 'one_hot_features': <tf.Tensor: shape=(7,), dtype=float32, numpy=array([0., 0., 1., 0., 0., 0., 0.], dtype=float32)>}, <tf.Tensor: shape=(), dtype=float32, numpy=0.4113175570964813>)
({'userId': <tf.Tensor: shape=(), dtype=int32, numpy=0>, 'locationId': <tf.Tensor: shape=(), dtype=int32, numpy=0>, 'numeric_features': <tf.Tensor: shape=(2,), dtype=float32

In [57]:
X_continuous_features

NameError: name 'X_continuous_features' is not defined

In [91]:
# Prepare the inputs
X_user = data['userId'].values
X_location = data['locationId'].values
X_continuous = X_continuous_features
X_one_hot = data[one_hot_weekdays].values

In [93]:
dataset = tf.data.Dataset.from_tensor_slices((X_user, X_location, X_continuous, X_one_hot))

In [95]:
def preprocess(user, loc, cont, one_hot):
    # Expand dimensions for scalar tensors (user and loc)
    user = tf.expand_dims(tf.cast(user, tf.float32), axis=-1)  # Shape: (1,)
    loc = tf.expand_dims(tf.cast(loc, tf.float32), axis=-1)    # Shape: (1,)
    user_embeds = user_embedding_layer(user)        # Convert userId to embeddings
    location_embeds = location_embedding_layer(loc)  # Convert locationId to embeddings

    cont = tf.expand_dims(cont, axis=0)  # Shape: (1, 3)
    one_hot = tf.expand_dims(one_hot, axis=0)  # Shape: (1, 7)
    cont = tf.cast(cont, tf.float32)                          # Shape: (3,)
    one_hot = tf.cast(one_hot, tf.float32)                    # Shape: (7,)

    # Concatenate all features
    combined = tf.concat([user_embeds, location_embeds, cont, one_hot], axis=-1)  # Final shape: (12,)
    return combined

In [97]:
final_dataset = dataset.map(preprocess)

In [99]:
next(iter(final_dataset))

<tf.Tensor: shape=(1, 26), dtype=float32, numpy=
array([[ 4.0756349e-02,  4.3033410e-02,  3.3013884e-02,  2.2738960e-02,
         4.2687681e-02, -3.0559147e-02,  1.9517552e-02, -3.7753902e-02,
        -1.2664914e-02,  2.1826532e-02,  1.7637733e-02, -3.3777677e-02,
         4.0508855e-02,  2.7414411e-04, -2.7805567e-03,  2.8349642e-02,
         2.7126130e-02,  6.5217394e-01,  7.3913044e-01,  0.0000000e+00,
         1.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00,  0.0000000e+00]], dtype=float32)>

In [256]:
# Combine all inputs into a TensorFlow dataset_batch
batch_size = 10 
dataset = tf.data.Dataset.from_tensor_slices((X_user, X_location, X_continuous, X_one_hot))
dataset = dataset.batch(batch_size)

In [ ]:
combined_X = tf.concat([tf.expand_dims(user_tensor, axis=-1), 
                        tf.expand_dims(loc_tensor, axis=-1), 
                        cont_tensor, 
                        one_hot_tensor], axis=-1)

In [266]:
np.concatenate((X_user, X_location, X_continuous, X_one_hot), axis=1)

AxisError: axis 1 is out of bounds for array of dimension 1

In [282]:
X_one_hot.shape

(3395, 7)

In [258]:
next(iter(dataset))

(<tf.Tensor: shape=(10,), dtype=int64, numpy=array([27, 27, 27, 27, 27, 27, 27, 27, 27, 27])>,
 <tf.Tensor: shape=(10,), dtype=int64, numpy=array([ 6,  6,  6,  6, 11,  2,  6,  6, 13, 21])>,
 <tf.Tensor: shape=(10, 3), dtype=float64, numpy=
 array([[0.02712613, 0.65217391, 0.73913043],
        [0.03919784, 0.73913043, 0.82608696],
        [0.08436613, 0.52173913, 0.69565217],
        [0.03179386, 0.82608696, 0.91304348],
        [0.00518077, 0.86956522, 0.91304348],
        [0.00741907, 0.60869565, 0.65217391],
        [0.01136249, 0.65217391, 0.65217391],
        [0.01807738, 0.86956522, 0.91304348],
        [0.00301793, 0.73913043, 0.7826087 ],
        [0.00679536, 0.7826087 , 0.7826087 ]])>,
 <tf.Tensor: shape=(10, 7), dtype=int64, numpy=
 array([[0, 1, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0, 0, 0],
 

In [238]:
# Function to replace userId and locationId with embeddings
def get_embeddings(user_ids, location_ids, continuous, one_hot):
    user_embeds = user_embedding_layer(user_ids)        # Convert userId to embeddings
    location_embeds = location_embedding_layer(location_ids)  # Convert locationId to embeddings

    # Ensure all tensors are of type float32
    continuous = tf.cast(continuous, tf.float32)
    one_hot = tf.cast(one_hot, tf.float32)
    
    # Concatenate embeddings and other features
    X_combined = tf.concat([user_embeds, location_embeds, continuous, one_hot], axis=1)
    return X_combined

In [240]:
# Map the function to process each batch
final_dataset = dataset.map(lambda user, loc, cont, one_hot: get_embeddings(user, loc, cont, one_hot))

In [248]:
dataset_iterator = iter(final_dataset)
batch = next(dataset_iterator)

InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_1_device_/job:localhost/replica:0/task:0/device:CPU:0}} Error in user-defined function passed to MapDataset:34 transformation with iterator: Iterator::Root::Prefetch::Map: indices[0] = 6 is not in [0, 0)
	 [[{{node location_embedding_1/GatherV2}}]] [Op:IteratorGetNext] name: 

In [250]:
user_embeds11 = user_embedding_layer(data['userId'])        # Convert userId to embeddings

InvalidArgumentError: Exception encountered when calling Embedding.call().

[1m{{function_node __wrapped__GatherV2_device_/job:localhost/replica:0/task:0/device:CPU:0}} indices[0] = 27 is not in [0, 0) [Op:GatherV2][0m

Arguments received by Embedding.call():
  • inputs=tf.Tensor(shape=(3395,), dtype=int64)

In [230]:
# an alternative way to embade. This one is not working either
def get_embeddings(user, loc, cont, one_hot):
    user_embedded = user_embedding_layer(user)  # Shape (batch_size, user_embedding_dim)
    loc_embedded = location_embedding_layer(loc)  # Shape (batch_size, location_embedding_dim)

    # Ensure that 'cont' is of type float32
    cont = tf.cast(cont, tf.float32)  # Convert 'cont' to float32
    
    # You can concatenate the embeddings with the continuous features and one-hot encodings
    embeddings = tf.concat([user_embedded, loc_embedded, cont], axis=-1)  # Shape (batch_size, user_embedding_dim + location_embedding_dim + cont_dim)
    
    return embeddings, one_hot

# Apply the map function
final_dataset = dataset.map(lambda user, loc, cont, one_hot: get_embeddings(user, loc, cont, one_hot))

In [232]:
final_dataset

<_MapDataset element_spec=(TensorSpec(shape=(None, 19), dtype=tf.float32, name=None), TensorSpec(shape=(None, 7), dtype=tf.int64, name=None))>

In [163]:
# Iterate through the dataset and print a sample batch
for X_minibatch in final_dataset.take(1):  # Take only the first mini-batch for testing
    #print("Mini-batch shape:", X_minibatch.shape)
    print(X_minibatch)

2024-12-17 19:25:12.783261: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: INVALID_ARGUMENT: Error in user-defined function passed to MapDataset:15 transformation with iterator: Iterator::Root::Prefetch::FiniteTake::Map: indices[0] = 27 is not in [0, 0)
	 [[{{node user_embedding_1/GatherV2}}]]


InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_1_device_/job:localhost/replica:0/task:0/device:CPU:0}} Error in user-defined function passed to MapDataset:15 transformation with iterator: Iterator::Root::Prefetch::FiniteTake::Map: indices[0] = 27 is not in [0, 0)
	 [[{{node user_embedding_1/GatherV2}}]] [Op:IteratorGetNext] name: 

In [97]:
user_input = tf.data.Dataset.from_tensor_slices(data['userId'].values)
location_input = tf.data.Dataset.from_tensor_slices(data['locationId'].values)
X_one_hot = tf.data.Dataset.from_tensor_slices(data[['Mon', 'Tues', 'Wed', 'Thurs', 'Fri', 'Sat', 'Sun']].values)
X_continuous = tf.data.Dataset.from_tensor_slices(X_continuous)

In [ ]:
def model(X_train, Y_train, X_test, Y_test, learning_rate = 0.0001,
          num_epochs = 1500, minibatch_size = 32, print_cost = True):
    """
    """

    train_acc = []
    test_acc = []
    costs = []

    parameters = initialize_parameters()

    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    W3 = parameters['W3']
    b3 = parameters['b3']

    # define the optimizer
    optimizer_defined = tf.keras.optimizers.Adam(learning_rate)

    # keep track of model accuracy
    test_accuracy = tf.keras.metrics.CategoricalAccuracy()
    train_accuracy = tf.keras.metrics.CategoricalAccuracy()

    # zip the data into a tuple of (input, expected label)
    dataset = tf.data.Dataset.zip((X_train, Y_train))
    test_dataset = tf.data.Dataset.zip((X_test, Y_test))

    # get the number of elements of dataset using the cardiality method
    m = dataset.cardinality().numpy()

    # form minibatches from the dataset
    minibatches = dataset.batch(minibatch_size).prefetch(8)
    test_minibatches = test_dataset.batch(minibatch_size).prefetch(8)

    for epoch in range(num_epochs):
        epoch_cost = 0

        # we need to restart from 0 when start a new epoch
        train_accuracy.reset_state()
        
        for (minibatch_X, minibatch_Y) in minibatches:
            with tf.GradientTape() as tape:
                # predict
                z3 = forward_propagation(tf.transpose(minibatch_X), parameters)

                # cost
                minibatch_cost = compute_cost(z3, tf.transpose(minibatch_Y))
                #print(minibatch_Y.shape, z3.shape)

            # update the accuracy of all batches
            train_accuracy.update_state(tf.transpose(z3), minibatch_Y)

            # now use the gradient tape to get the gradients and update the parameters
            trainable_variables = [W1, b1, W2, b2, W3, b3]
            grads = tape.gradient(minibatch_cost, trainable_variables)
            optimizer_defined.apply_gradients(zip(grads, trainable_variables))

            # add all the costs
            epoch_cost += minibatch_cost

        # we devide the cost of each epoch by number of samples to get the cost per sample
        epoch_cost /= m

        # print cost every 10 epoch
        if print_cost == True and epoch % 10 == 0:
            print("cost after epoch %i: %f" % (epoch, epoch_cost))
            print("train accuracy:", train_accuracy.result())

            for (minibatch_X, minibatch_Y) in test_minibatches:
                Z3 = forward_propagation(tf.transpose(minibatch_X), parameters)
                test_accuracy.update_state(tf.transpose(Z3), minibatch_Y)
            print("Test_accuracy:", test_accuracy.result())

            costs.append(epoch_cost)
            train_acc.append(train_accuracy.result())
            test_acc.append(test_accuracy.result())
            test_accuracy.reset_state() # this resets every 10 epoch

    return parameters, costs, train_acc, test_acc

In [105]:
X_continuous_iter = iter(X_continuous)
for i in range(10):
    print(next(X_continuous_iter))

tf.Tensor([0.02712613 0.65217391 0.73913043], shape=(3,), dtype=float64)
tf.Tensor([0.03919784 0.73913043 0.82608696], shape=(3,), dtype=float64)
tf.Tensor([0.08436613 0.52173913 0.69565217], shape=(3,), dtype=float64)
tf.Tensor([0.03179386 0.82608696 0.91304348], shape=(3,), dtype=float64)
tf.Tensor([0.00518077 0.86956522 0.91304348], shape=(3,), dtype=float64)
tf.Tensor([0.00741907 0.60869565 0.65217391], shape=(3,), dtype=float64)
tf.Tensor([0.01136249 0.65217391 0.65217391], shape=(3,), dtype=float64)
tf.Tensor([0.01807738 0.86956522 0.91304348], shape=(3,), dtype=float64)
tf.Tensor([0.00301793 0.73913043 0.7826087 ], shape=(3,), dtype=float64)
tf.Tensor([0.00679536 0.7826087  0.7826087 ], shape=(3,), dtype=float64)


In [77]:
data['userId'][150]

27

In [11]:
def preprocess(data):
    # Label Encoding for userId and locationId
    user_encoder = LabelEncoder()
    location_encoder = LabelEncoder()

    # conert them into numbers ranging from 0 to user-1
    data['userId'] = user_encoder.fit_transform(data['userId'])
    data['locationId'] = location_encoder.fit_transform(data['locationId'])

    # Convert `userId` and `locationId` columns to TensorFlow tensors
    user_input = tf.convert_to_tensor(data['userId'].values, dtype=tf.int32)  # Shape: (num_samples,)
    location_input = tf.convert_to_tensor(data['locationId'].values, dtype=tf.int32)  # Shape: (num_samples,)

    # the number of unique user IDs
    num_users = len(user_encoder.classes_)
    num_locations = len(location_encoder.classes_)


    # Define embedding layers
    user_embedding_dim = 8  # You can choose any dimension
    location_embedding_dim = 8  # You can choose any dimension

    user_embedding_layer = tf.keras.layers.Embedding(input_dim=num_users, output_dim=user_embedding_dim, name='user_embedding')
    location_embedding_layer = tf.keras.layers.Embedding(input_dim=num_locations, output_dim=location_embedding_dim, name='location_embedding')

    # Generate embeddings
    user_embedding = user_embedding_layer(user_input)  # Shape: (num_samples, user_embedding_dim)
    location_embedding = location_embedding_layer(location_input)  # Shape: (num_samples, location_embedding_dim)

    # Flatten embeddings for concatenation
    user_embedding = tf.reshape(user_embedding, [-1, user_embedding_dim])
    location_embedding = tf.reshape(location_embedding, [-1, location_embedding_dim])

    X_one_hot = tf.convert_to_tensor(data[['Mon', 'Tues', 'Wed', 'Thurs', 'Fri', 'Sat', 'Sun']].values, dtype=tf.float32)

    ### normalize the continuous variables
    # List of continuous features
    continuous_features = ['chargeTimeHrs', 'startTime', 'endTime']

    # Normalize the features
    scaler = MinMaxScaler()
    X_continuous = scaler.fit_transform(data[continuous_features])
    X_continuous = tf.convert_to_tensor(X_continuous, dtype=tf.float32)

    # Final input tensor
    X = tf.concat([user_embedding, location_embedding, X_one_hot, X_continuous], axis=1) 

    # Now, userIds will be integers from 0 to (num_users - 1)
    return X

In [17]:
# Create a TensorFlow dataset
dataset = tf.data.Dataset.from_tensor_slices(preprocess(data))
#dataset = dataset.map(lambda row: preprocess(row))

In [21]:
next(iter(dataset))

<tf.Tensor: shape=(26,), dtype=float32, numpy=
array([-0.01510651,  0.00575405,  0.00688646,  0.04031584, -0.00453073,
       -0.01202203,  0.0462589 ,  0.02595312, -0.04877306,  0.006142  ,
        0.0010676 , -0.01837022, -0.00747188, -0.04814326,  0.02399974,
        0.04982344,  0.        ,  1.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.02712613,  0.65217394,
        0.73913044], dtype=float32)>

In [43]:
# Read only the first 30 rows
df = pd.read_csv(data, nrows=30)

TypeError: argument of type 'method' is not iterable

In [35]:
df

,sessionId,kwhTotal,dollars,created,ended,startTime,endTime,chargeTimeHrs,weekday,platform,...,managerVehicle,facilityType,Mon,Tues,Wed,Thurs,Fri,Sat,Sun,reportedZip
0,1366563,7.78,0.00,0014-11-18 15:40:26,0014-11-18 17:11:04,15,17,1.510556,Tue,android,...,0,3,0,1,0,0,0,0,0,0
1,3075723,9.74,0.00,0014-11-19 17:40:26,0014-11-19 19:51:04,17,19,2.177222,Wed,android,...,0,3,0,0,1,0,0,0,0,0
2,4228788,6.76,0.58,0014-11-21 12:05:46,0014-11-21 16:46:04,12,16,4.671667,Fri,android,...,0,3,0,0,0,0,1,0,0,0
3,3173284,6.17,0.00,0014-12-03 19:16:12,0014-12-03 21:02:18,19,21,1.768333,Wed,android,...,0,3,0,0,1,0,0,0,0,0
4,3266500,0.93,0.00,0014-12-11 20:56:11,0014-12-11 21:14:06,20,21,0.298611,Thu,android,...,0,3,0,0,0,1,0,0,0,0
5,4099366,2.14,0.00,0014-12-12 14:38:44,0014-12-12 15:04:04,14,15,0.422222,Fri,android,...,0,3,0,0,0,0,1,0,0,0
6,5084244,0.30,0.00,0014-12-12 15:08:40,0014-12-12 15:47:04,15,15,0.640000,Fri,android,...,0,3,0,0,0,0,1,0,0,0
7,2948436,1.82,0.00,0014-12-17 20:30:25,0014-12-17 21:31:04,20,21,1.010833,Wed,android,...,0,3,0,0,1,0,0,0,0,0
8,3515913,0.81,0.00,0014-12-18 17:53:19,0014-12-18 18:04:04,17,18,0.179167,Thu,android,...,0,3,0,0,0,1,0,0,0,0
9,8490014,1.98,0.00,0014-12-18 18:06:49,0014-12-18 18:30:05,18,18,0.387778,Thu,android,...,0,3,0,0,0,1,0,0,0,0
